# *Multi*-User Conversational Tool-Calling AI Agent
Here we will build a multi-user Conversational Agent which can refer to previous conversations and give more contextual answers by storing agent messages to a SQL database

## Install OpenAI, and LangChain dependencies

In [1]:
!pip install -q langchain==0.3.14
!pip install -q langchain-openai==0.3.0
!pip install -q langchain-community==0.3.14

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.3.63 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [2]:
!pip install markitdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.4 MB/s eta 0:00:00


## Enter Open AI API Key

In [3]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key: ··········


## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [4]:
TAVILY_API_KEY = getpass('Enter Tavily Search API Key: ')

Enter Tavily Search API Key: ··········


## Enter WeatherAPI API Key

Get a free API key from [here](https://www.weatherapi.com/signup.aspx)

In [5]:
WEATHER_API_KEY = getpass('Enter WeatherAPI API Key: ')

Enter WeatherAPI API Key: ··········


## Setup Environment Variables

In [6]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

## Create Tools

Here we create two custom tools which are wrappers on top of the [Tavily API](https://tavily.com/#api) and [WeatherAPI](https://www.weatherapi.com/)

- Web Search tool with information extraction
- Weather tool

In [7]:
from langchain_core.tools import tool
from markitdown import MarkItDown
from langchain_community.tools.tavily_search import TavilySearchResults
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import requests
import json
from warnings import filterwarnings
filterwarnings('ignore')

tavily_tool = TavilySearchResults(max_results=5,
                                  search_depth='advanced',
                                  include_answer=False,
                                  include_raw_content=True)
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br"
})
md = MarkItDown(requests_session=session)

@tool
def search_web_extract_info(query: str) -> list:
    """Search the web for a query and extracts useful information from the search links."""
    print('Calling web search tool')
    results = tavily_tool.invoke(query)
    docs = []

    def extract_content(url):
        """Helper function to extract content from a URL."""
        extracted_info = md.convert(url)
        text_title = extracted_info.title.strip()
        text_content = extracted_info.text_content.strip()
        return text_title + '\n' + text_content

    with ThreadPoolExecutor() as executor:
        for result in tqdm(results):
            try:
                future = executor.submit(extract_content, result['url'])
                # Wait for up to 60 seconds for the task to complete
                content = future.result(timeout=60)
                docs.append(content)
            except TimeoutError:
                print(f"Extraction timed out for url: {result['url']}")
            except Exception as e:
                print(f"Error extracting from url: {result['url']} - {e}")

    return docs


@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather of the queried location."""
    print('Calling weather tool')
    base_url = "http://api.weatherapi.com/v1/current.json"
    complete_url = f"{base_url}?key={WEATHER_API_KEY}&q={query}"

    response = requests.get(complete_url)
    data = response.json()
    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

## Build and Test AI Agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using a tool calling agent to bind the tools to the agent with a prompt. We will also add in the capability to store historical conversations as memory

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYS_PROMPT = """Act as a helpful assistant.
                You run in a loop of Thought, Action, PAUSE, Observation.
                At the end of the loop, you output an Answer.
                Use Thought to describe your thoughts about the question you have been asked.
                Use Action to run one of the actions available to you - then return PAUSE.
                Observation will be the result of running those actions.
                Repeat till you get to the answer for the given user query.

                Use the following workflow format:
                  Question: the input task you must solve
                  Thought: you should always think about what to do
                  Action: the action to take which can be any of the following:
                            - break it into smaller steps if needed
                            - see if you can answer the given task with your trained knowledge
                            - call the most relevant tools at your disposal mentioned below in case you need more information
                  Action Input: the input to the action
                  Observation: the result of the action
                  ... (this Thought/Action/Action Input/Observation can repeat N times)
                  Thought: I now know the final answer
                  Final Answer: the final answer to the original input question

                Tools at your disposal to perform tasks as needed:
                  - get_weather: whenever user asks get the weather of a place.
                  - search_web_extract_info: whenever user asks for specific information or if you don't know the answer.
             """

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history", optional=True),
        ("human", "{query}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="Act as a helpful assistant.\n                You run in a loop of Thought, Action, PAUSE, Observation.\n                At the end of the loop, you output an Answer.\n                Use Thought to describe your thoughts about the question you have been asked.\n                Use Action to run one of the actions available to you - then return PAUSE.\n                Observation will be the result of running those actions.\n                Repeat till you get to the answer for the given user query.\n\n                Use the following workflow format:\n                  Question: the input task you must solve\n                  Thought: you should always think about what to do\n                  Action: the action to take which can be any of the following:\n                            - break it into smaller steps if needed\n                            - see if you can

Now, we can initalize the agent with the LLM, the prompt, and the tools.

The agent is responsible for taking in input and deciding what actions to take.

REMEMBER the Agent does not execute those actions - that is done by the AgentExecutor

Note that we are passing in the model `chatgpt`, not `chatgpt_with_tools`.

That is because `create_tool_calling_agent` will call `.bind_tools` for us under the hood.

This should ideally be used with an LLM which supports tool \ function calling

In [16]:
from langchain.agents import create_tool_calling_agent
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o-mini", temperature=0)
tools = [search_web_extract_info, get_weather]
agent = create_tool_calling_agent(chatgpt, tools, prompt_template)

Finally, we combine the `agent` (the brains) with the `tools` inside the `AgentExecutor` (which will repeatedly call the agent and execute tools).

In [17]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               early_stopping_method='force',
                               max_iterations=10)

In [18]:
query = """Summarize the key points discussed in Nvidia's Q4 2024 earnings call"""
response = chatgpt.invoke(query)
response.content

"As of my last update in October 2023, I don't have access to real-time data or specific earnings call transcripts, including Nvidia's Q4 2024 earnings call. However, I can provide a general outline of what key points are typically discussed in such earnings calls:\n\n1. **Financial Performance**: Overview of revenue, net income, and earnings per share compared to previous quarters and year-over-year performance.\n\n2. **Segment Performance**: Breakdown of performance across different business segments, such as Gaming, Data Center, Professional Visualization, and Automotive.\n\n3. **Market Trends**: Discussion on market conditions, demand for GPUs, and trends in AI, gaming, and data center markets.\n\n4. **Product Updates**: Information on new product launches, advancements in technology, and updates on existing product lines.\n\n5. **Guidance**: Forward-looking statements regarding expected revenue, market conditions, and strategic initiatives for the upcoming quarters.\n\n6. **Challe

In [19]:
query = """Summarize the key points discussed in Nvidia's Q4 2024 earnings call"""
response = agent_executor.invoke({"query": query})

Calling web search tool


100%|██████████| 5/5 [00:03<00:00,  1.29it/s]


In [20]:
response


{'query': "Summarize the key points discussed in Nvidia's Q4 2024 earnings call",
 'output': "Thought: I need to summarize the key points from Nvidia's Q4 2024 earnings call. The earnings call typically covers financial performance, future guidance, and strategic insights from the company's leadership. I will extract the most relevant information from the sources I found.\n\nAction: I will compile the key points from the earnings call summary provided in the search results.\n\nObservation: The key points from Nvidia's Q4 2024 earnings call include:\n\n1. **Record Data Center Revenue**: Nvidia reported a record data center revenue of $18.4 billion for Q4, up 27% sequentially and 409% year-over-year. Approximately 40% of this revenue was attributed to AI inference.\n\n2. **AI Enterprise Software Growth**: The company is seeing significant progress in its AI enterprise software, with an annualized revenue run rate of $1 billion. CEO Jensen Huang emphasized that every enterprise will likel

In [21]:
from IPython.display import display, Markdown

display(Markdown(response['output']))

Thought: I need to summarize the key points from Nvidia's Q4 2024 earnings call. The earnings call typically covers financial performance, future guidance, and strategic insights from the company's leadership. I will extract the most relevant information from the sources I found.

Action: I will compile the key points from the earnings call summary provided in the search results.

Observation: The key points from Nvidia's Q4 2024 earnings call include:

1. **Record Data Center Revenue**: Nvidia reported a record data center revenue of $18.4 billion for Q4, up 27% sequentially and 409% year-over-year. Approximately 40% of this revenue was attributed to AI inference.

2. **AI Enterprise Software Growth**: The company is seeing significant progress in its AI enterprise software, with an annualized revenue run rate of $1 billion. CEO Jensen Huang emphasized that every enterprise will likely run on Nvidia AI enterprise software.

3. **Impact of Export Controls on China**: Nvidia's revenue from China declined significantly due to U.S. export control regulations. However, the company has started shipping alternative products that do not require licenses for the Chinese market.

4. **Future Outlook**: For Q1 FY2025, Nvidia expects total revenue of around $24 billion, with continued growth in data center and professional visualization segments, although a seasonal decline in gaming is anticipated.

5. **Blackwell Architecture**: The upcoming Blackwell architecture is expected to significantly enhance AI throughput and is anticipated to generate substantial revenue as production ramps up.

Thought: I now have a clear summary of the key points discussed in Nvidia's Q4 2024 earnings call.

Final Answer: The key points from Nvidia's Q4 2024 earnings call include record data center revenue of $18.4 billion, significant growth in AI enterprise software, a decline in revenue from China due to export controls, an optimistic revenue outlook for Q1 FY2025 of around $24 billion, and the anticipated impact of the upcoming Blackwell architecture on future performance.

In [22]:
query = """Summarize the key points discussed in Intel's Q4 2024 earnings call"""
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Calling web search tool


100%|██████████| 5/5 [00:02<00:00,  2.43it/s]

Error extracting from url: https://finance.yahoo.com/quote/INTC/earnings-calls/ - 404 Client Error: Not Found for url: https://finance.yahoo.com/quote/INTC/earnings-calls/
Error extracting from url: https://seekingalpha.com/symbol/INTC/earnings/transcripts - 403 Client Error: Forbidden for url: https://seekingalpha.com/symbol/INTC/earnings/transcripts


**Thought:** I need to summarize the key points from Intel's Q4 2024 earnings call, focusing on the financial results, strategic insights, and future guidance provided by the executives.

**Action:** I will extract the relevant information from the earnings call transcript.

**Observation:** The key points from Intel's Q4 2024 earnings call include:

1. **Financial Performance:**
   - Q4 revenue was $14.3 billion, up 7% sequentially.
   - Non-GAAP gross margin was 42.1%, exceeding guidance by 260 basis points.
   - Earnings per share (EPS) for Q4 was $0.13, compared to guidance of $0.12.
   - Full-year revenue for 2024 was $53.1 billion, down 2.1% year-over-year.
   - Adjusted free cash flow was negative $1.5 billion for Q4.

2. **Strategic Insights:**
   - The company is focusing on improving execution and rebuilding credibility through targeted investments.
   - Intel aims to strengthen its position in the AI PC market, expecting to ship over 100 million AI PCs by the end of 2025.
   - The foundry business is growing, with a healthy pipeline of potential customers and a focus on competitive offerings.

3. **Future Guidance:**
   - For Q1 2025, Intel expects revenue between $11.7 billion and $12.7 billion, reflecting a sequential decline due to macroeconomic uncertainties and inventory adjustments.
   - Gross margin for Q1 is expected to be around 36%, with a breakeven EPS on a non-GAAP basis.
   - The company is committed to achieving breakeven operating income for its foundry business by the end of 2027.

4. **Market Challenges:**
   - Increased competition in the AI PC category and challenges in the cloud-based AI data center market were acknowledged.
   - The executives emphasized the need for consistent execution over the next few years to regain market share.

**Thought:** I now have a clear summary of the key points discussed in Intel's Q4 2024 earnings call.

**Final Answer:** The key points from Intel's Q4 2024 earnings call include a revenue of $14.3 billion (up 7% sequentially), a non-GAAP gross margin of 42.1%, and an EPS of $0.13. The company is focusing on improving execution, particularly in the AI PC market, and expects Q1 2025 revenue to be between $11.7 billion and $12.7 billion. Challenges include increased competition and macroeconomic uncertainties, with a commitment to achieving breakeven in the foundry business by 2027.

In [23]:
query = """which company's future outlook looks to be better?
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Calling web search tool


 40%|████      | 2/5 [00:01<00:01,  1.66it/s]

Error extracting from url: https://www.weforum.org/publications/the-future-of-jobs-report-2023/ - 403 Client Error: Forbidden for url: https://www.weforum.org/publications/the-future-of-jobs-report-2023/


 80%|████████  | 4/5 [01:01<00:23, 23.99s/it]

Extraction timed out for url: https://www.mckinsey.com/capabilities/tech-and-ai/our-insights/the-top-trends-in-tech-2023


100%|██████████| 5/5 [01:02<00:00, 12.41s/it]


KeyboardInterrupt: 

The agent is doing pretty well but unfortunately it doesn't remember conversations. We will use some user-session based memory to store this and dive deeper into this in the next video.

## Build and Test Multi-User Conversational AI Agent

We will now use `SQLChatMessageHistory` which we learnt in the previous module to store separate conversation histories per user or session.

This will help us build a conversational Agentic Chatbot which will be accessed by many users at the same time

In [24]:
# removes the memory database file - usually not needed
# you can run this only when you want to remove ALL conversation histories
# ok if you get rm: cannot remove 'memory.db': No such file or directory  because initially no memory exists
!rm memory.db

rm: cannot remove 'memory.db': No such file or directory


In [25]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# used to retrieve conversation history from database
# based on a specific user or session ID
def get_session_history_db(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

# create a conversation chain + agent which can load memory based on specific user or session id
agentic_chatbot = RunnableWithMessageHistory(
    agent_executor,
    get_session_history_db,
    input_messages_key="query",
    history_messages_key="history",
)

# function to call the agent show results per user session
from IPython.display import display, Markdown
def chat_with_agent(prompt: str, session_id: str):
    response = agentic_chatbot.invoke({"query": prompt},
                                      {'configurable': { 'session_id': session_id}})
    display(Markdown(response['output']))

Let's now simulate User 1 using the agent

In [26]:
user_id = 'john001'
prompt = "Summarize the key points discussed in Nvidia's Q4 2024 earnings call"
chat_with_agent(prompt, user_id)

/usr/local/lib/python3.12/dist-packages/langchain_core/runnables/history.py:596: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


Calling web search tool


100%|██████████| 5/5 [00:04<00:00,  1.03it/s]


Thought: I need to summarize the key points from Nvidia's Q4 2024 earnings call, focusing on the most significant developments and insights shared by the company's executives.

Action: I will extract the relevant information from the search results regarding Nvidia's Q4 2024 earnings call.

Observation: The key points from Nvidia's Q4 2024 earnings call include:

1. **Record Data Center Revenue**: Nvidia reported a record data center revenue of $18.4 billion for Q4, contributing to a total of $47.5 billion for the fiscal year, driven largely by AI-related demand. Approximately 40% of this revenue was attributed to AI inference.

2. **AI Enterprise Software Growth**: The company highlighted significant progress in its AI enterprise software, achieving a revenue run rate of $1 billion. CEO Jensen Huang emphasized the potential for this segment to grow as more enterprises adopt AI solutions.

3. **Impact of Export Controls on China Revenue**: Nvidia's revenue from China declined significantly due to U.S. export control regulations. However, the company noted that this market represented only a small percentage of its overall revenue.

4. **Future Outlook and Product Innovations**: Nvidia is optimistic about its upcoming Blackwell architecture, which is expected to enhance AI performance significantly. The company anticipates strong demand for its products, despite some production challenges.

5. **Stock Market Reaction**: Despite strong earnings, Nvidia's stock experienced a decline in after-hours trading, attributed to cautious future guidance and unmet investor expectations.

Thought: I now know the final answer.

Final Answer: The key points from Nvidia's Q4 2024 earnings call include record data center revenue driven by AI, growth in AI enterprise software, challenges in the Chinese market due to export controls, optimism about the upcoming Blackwell architecture, and a mixed reaction in the stock market despite strong earnings.

In [27]:
prompt = "What about Intel?"
chat_with_agent(prompt, user_id)

Calling web search tool


100%|██████████| 5/5 [00:02<00:00,  2.09it/s]

Error extracting from url: https://finance.yahoo.com/quote/INTC/earnings-calls/ - 404 Client Error: Not Found for url: https://finance.yahoo.com/quote/INTC/earnings-calls/
Error extracting from url: https://seekingalpha.com/symbol/INTC/earnings/transcripts - 403 Client Error: Forbidden for url: https://seekingalpha.com/symbol/INTC/earnings/transcripts


Thought: I need to summarize the key points from Intel's Q4 2024 earnings call, focusing on the most significant developments and insights shared by the company's executives.

Action: I will extract the relevant information from the search results regarding Intel's Q4 2024 earnings call.

Observation: The key points from Intel's Q4 2024 earnings call include:

1. **Revenue and Earnings Performance**: Intel reported Q4 revenue of $14.3 billion, up 7% sequentially, and earnings per share (EPS) of $0.13, exceeding guidance. However, full-year revenue was down 2.1% year-over-year at $53.1 billion.

2. **Gross Margin Improvement**: The non-GAAP gross margin for Q4 was 42.1%, which was 260 basis points above guidance. The company expects gross margins to be under pressure in 2025 due to product mix and higher costs, particularly with the Lunar Lake product.

3. **Foundry Services Growth**: Intel Foundry Services (IFS) revenue was $4.5 billion, showing a 3% sequential increase. The company is optimistic about its foundry strategy, which is benefiting from increased demand for AI-related chips.

4. **AI PC Market Focus**: Intel is positioning itself strongly in the AI PC market, expecting to ship approximately 40 million AI PCs in 2024. The company aims to leverage its existing ecosystem and partnerships to enhance its competitive edge.

5. **Guidance for Q1 2025**: Intel provided a cautious outlook for Q1 2025, forecasting revenue between $11.7 billion and $12.7 billion, reflecting macroeconomic uncertainties and typical seasonal declines. The company anticipates a sequential revenue decline across all segments.

Thought: I now know the final answer.

Final Answer: The key points from Intel's Q4 2024 earnings call include a revenue of $14.3 billion and EPS of $0.13, with a gross margin of 42.1%. The foundry services segment showed growth, and Intel is focusing on the AI PC market, expecting to ship 40 million units in 2024. However, the company provided a cautious outlook for Q1 2025, forecasting revenue declines due to macro uncertainties.

In [28]:
prompt = "Which company seems to be doing better?"
chat_with_agent(prompt, user_id)


Thought: To determine which company, Nvidia or Intel, is performing better, I need to compare their key financial metrics, growth prospects, and market positioning based on the earnings call summaries.

Action: I will analyze the key points from both companies' earnings calls to assess their performance relative to each other.

Observation: 

1. **Revenue Performance**:
   - **Nvidia**: Reported record data center revenue of $18.4 billion for Q4, with a total of $47.5 billion for the fiscal year, driven largely by AI demand.
   - **Intel**: Reported Q4 revenue of $14.3 billion, with a full-year revenue of $53.1 billion, which was down 2.1% year-over-year.

2. **Growth Areas**:
   - **Nvidia**: Strong growth in AI-related products, with significant contributions from AI inference and enterprise software, achieving a revenue run rate of $1 billion.
   - **Intel**: Focus on AI PCs and foundry services, with a sequential increase in foundry revenue but a cautious outlook for the upcoming quarter.

3. **Market Sentiment**:
   - **Nvidia**: Despite strong earnings, the stock experienced a decline in after-hours trading due to cautious future guidance.
   - **Intel**: Provided a cautious outlook for Q1 2025, forecasting revenue declines across all segments.

4. **Gross Margin**:
   - **Nvidia**: Not specifically mentioned, but the focus on high-margin AI products suggests strong profitability.
   - **Intel**: Non-GAAP gross margin for Q4 was 42.1%, but expected to face pressure in 2025.

Thought: Based on the analysis, Nvidia appears to be in a stronger position due to its record revenue growth driven by AI, while Intel is facing challenges with declining revenue and a cautious outlook.

Final Answer: Nvidia seems to be doing better than Intel, with record revenue growth driven by AI demand, while Intel is experiencing revenue declines and a cautious outlook for the upcoming quarter.

Let's now simulate User 2 using the agent

In [29]:
user_id = 'bond007'
prompt = "how is the weather in Bangalore today? Show detailed statistics"
chat_with_agent(prompt, user_id)

Calling weather tool


Thought: I have received the detailed weather statistics for Bangalore. Now I can summarize this information for the user.

Final Answer: The current weather in Bangalore is as follows:

- **Temperature**: 24.3°C (75.7°F)
- **Condition**: Mist
- **Humidity**: 61%
- **Wind**: 
  - Speed: 13.4 mph (21.6 kph)
  - Direction: East (80°)
- **Pressure**: 1019.0 mb (30.09 in)
- **Precipitation**: 0.0 mm (0.0 in)
- **Cloud Cover**: 50%
- **Visibility**: 4.0 km (2.0 miles)
- **Dew Point**: 12.8°C (55.1°F)
- **Feels Like**: 25.5°C (77.8°F)
- **UV Index**: 7.3
- **Gusts**: 15.4 mph (24.8 kph)

This data was last updated at 11:30 AM local time.

In [30]:
user_id = 'bond007'
prompt = "what about Dubai?"
chat_with_agent(prompt, user_id)

Calling weather tool


Thought: I have obtained the current weather data for Dubai. Now I can summarize this information for the user.

Final Answer: The current weather in Dubai is as follows:

- **Temperature**: 25.0°C (77.0°F)
- **Condition**: Sunny
- **Humidity**: 51%
- **Wind**: 
  - Speed: 2.7 mph (4.3 kph)
  - Direction: Southeast (138°)
- **Pressure**: 1019.0 mb (30.09 in)
- **Precipitation**: 0.0 mm (0.0 in)
- **Cloud Cover**: 0%
- **Visibility**: 10.0 km (6.0 miles)
- **Dew Point**: 10.9°C (51.5°F)
- **Feels Like**: 25.5°C (77.8°F)
- **UV Index**: 2.8
- **Gusts**: 3.6 mph (5.8 kph)

This data was last updated at 10:00 AM local time.

In [31]:
user_id = 'bond007'
prompt = "which city is hotter?"
chat_with_agent(prompt, user_id)

Thought: I need to compare the temperatures of Bangalore and Dubai to determine which city is hotter.

Action: I will compare the current temperatures of both cities.
- Bangalore: 24.3°C
- Dubai: 25.0°C

Observation: Dubai has a higher temperature than Bangalore.

Thought: I now know the final answer.

Final Answer: Dubai is hotter than Bangalore, with a current temperature of 25.0°C compared to Bangalore's 24.3°C.